#Linear Regression

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import zipfile
import matplotlib.pyplot as plt

drive.mount('/content/drive', force_remount=True)
# CORE_PATH = '/content/drive/My Drive/babe/GenHack'
CORE_PATH = '/content/drive/MyDrive/GenHack'


Mounted at /content/drive


In [ ]:
#by day regression
df_lr = pd.read_csv('/content/drive/MyDrive/GenHack/df_lr1201.csv')

In [ ]:
!pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import statsmodels.api as sm

# --- CLEAN DATA ---
df_lr_cleaned = df_lr.dropna(
    subset=['diff_in_temp', 'mean_NDVI_transformed', 'HGHT', 'is_summer', 'STAID', 'DATE_x']
).copy()

X = df_lr_cleaned[['mean_NDVI_transformed', 'HGHT', 'is_summer']]
X['NVDI_squared'] = X['mean_NDVI_transformed'] ** 2
#x1: mean_NDVI_transformed, x2: HGHT, x3: is_summer, x4: NVDI_squared
y = df_lr_cleaned['diff_in_temp']

# --- TRAIN–TEST SPLIT ---
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, df_lr_cleaned.index, test_size=0.2, random_state=42
)

# --- SCALE FEATURES ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- SKLEARN REGRESSION ---
model = LinearRegression()
model.fit(X_train_scaled, y_train)

print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

# ============================================================
# --- PREP GROUPING VARIABLES FOR CLUSTERED SE ---
# Avoid dtype issues by converting to int-coded categories
# ============================================================

station_ids = df_lr_cleaned.loc[idx_train, 'STAID']
date_ids = df_lr_cleaned.loc[idx_train, 'DATE_x']

# Convert to safe integer codes
station_groups = station_ids.astype("category").cat.codes.astype("int64")
date_groups = date_ids.astype("category").cat.codes.astype("int64")

# --- DESIGN MATRIX FOR STATSMODELS ---
X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_train_sm = sm.add_constant(X_train_df)

# ============================================================
# --- FIT MODEL WITH CLUSTERED STANDARD ERRORS ---
# One-way clustering: by station
# ============================================================
ols_cluster_station = sm.OLS(y_train, X_train_sm).fit(
    cov_type="cluster",
    cov_kwds={"groups": station_groups}
)

# ============================================================
# --- TWO-WAY CLUSTERING (station + date) ---
# Statsmodels: pass Nx2 matrix
# ============================================================
groups_two_way = np.column_stack([station_groups, date_groups])

ols_cluster_tw = sm.OLS(y_train, X_train_sm).fit(
    cov_type="cluster",
    cov_kwds={"groups": groups_two_way}
)

# --- Print both for comparison ---
print("\n=== OLS with Station Clustering ===")
print(ols_cluster_station.summary())

print("\n=== OLS with Two-Way Clustering (Station + Date) ===")
print(ols_cluster_tw.summary())

/tmp/ipython-input-166593003.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['NVDI_squared'] = X['mean_NDVI_transformed'] ** 2


Coefficients: [-1.45692278  0.77772055  0.14007904  1.33675371]
Intercept: 1.5366082923614992

=== OLS with Station Clustering ===
                            OLS Regression Results                            
Dep. Variable:           diff_in_temp   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     12.48
Date:                Wed, 03 Dec 2025   Prob (F-statistic):           1.82e-06
Time:                        05:25:22   Log-Likelihood:                -79582.
No. Observations:               39458   AIC:                         1.592e+05
Df Residuals:                   39453   BIC:                         1.592e+05
Df Model:                           4                                         
Covariance Type:              cluster                                         
                            coef    std err          z      P>|z|      [0.025  

In [ ]:
#by quarter regression
df_temp = pd.read_csv('/content/drive/MyDrive/GenHack/temp_diff_dpvar.csv')

df_temp['DATE'] = pd.to_datetime(df_temp['DATE'])
df_temp['quarter_idx'] = df_temp['DATE'].dt.year * 100 + np.where(
    df_temp['DATE'].dt.month <= 3,
    1,
    np.where(
        df_temp['DATE'].dt.month <= 6,
        2,
        np.where(
            df_temp['DATE'].dt.month <= 9,
            3,
            4
        )
    )
)

df_temp_quarter = df_temp.groupby(
    by = ['STAID', 'quarter_idx']
)[['HGHT', 'diff_in_temp']].mean().reset_index() # Corrected column selection and added .mean() for aggregation
df_temp_quarter = df_temp_quarter[df_temp_quarter['diff_in_temp'].notna()]
df_temp_quarter['if_summer'] = np.where(
    df_temp_quarter['quarter_idx'] % 10 == 3,
    1,
    0
)
df_temp_quarter['NVDI_squared'] = df_temp_quarter['diff_in_temp'] ** 2


ndvi_df = pd.read_csv('/content/drive/MyDrive/GenHack/vegetation_around_stations.csv')
ndvi_df['quarter_idx'] = (ndvi_df['DATE'] // 10000) * 100 + np.where(
   (ndvi_df['DATE'] // 100) - (ndvi_df['DATE'] // 10000) * 100 <= 3,
   1,
   np.where(
       (ndvi_df['DATE'] // 100) - (ndvi_df['DATE'] // 10000) * 100 <= 6,
       2,
       np.where(
           (ndvi_df['DATE'] // 100) - (ndvi_df['DATE'] // 10000) * 100 <= 9,
           3,
           4
       )
   )
)
ndvi_df['STAID'] = ndvi_df['STAID'].astype(str)

ndvi_df['STAID'] = ndvi_df['STAID'].astype(int)
df_lr_qt = pd.merge(
    df_temp_quarter,
    ndvi_df[['STAID', 'quarter_idx', 'mean_NDVI_transformed']],
    on = ['STAID', 'quarter_idx'],
    how = 'left'
)
df_lr_qt = df_lr_qt[df_lr_qt['mean_NDVI_transformed'].notna()]

In [ ]:
# --- CLEAN DATA ---
df_lr_cleaned_qr = df_lr_qt.dropna(
    subset=['diff_in_temp', 'mean_NDVI_transformed', 'HGHT', 'if_summer', 'STAID', 'quarter_idx']
).copy()

X = df_lr_cleaned_qr[['mean_NDVI_transformed', 'HGHT', 'if_summer']]
X['NVDI_squared'] = X['mean_NDVI_transformed'] ** 2
#x1: mean_NDVI_transformed, x2: HGHT, x3: is_summer, x4: NVDI_squared
y = df_lr_cleaned_qr['diff_in_temp']

# --- TRAIN–TEST SPLIT ---
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, df_lr_cleaned_qr.index, test_size=0.2, random_state=42
)

# --- SCALE FEATURES ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- SKLEARN REGRESSION ---
model = LinearRegression()
model.fit(X_train_scaled, y_train)

print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

# ============================================================
# --- PREP GROUPING VARIABLES FOR CLUSTERED SE ---
# Avoid dtype issues by converting to int-coded categories
# ============================================================

station_ids = df_lr_cleaned_qr.loc[idx_train, 'STAID']
date_ids = df_lr_cleaned_qr.loc[idx_train, 'quarter_idx']


# Convert to safe integer codes
station_groups = station_ids.astype("category").cat.codes.astype("int64")
date_groups = date_ids.astype("category").cat.codes.astype("int64")

# --- DESIGN MATRIX FOR STATSMODELS ---
X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_train_sm = sm.add_constant(X_train_df)

# ============================================================
# --- FIT MODEL WITH CLUSTERED STANDARD ERRORS ---
# One-way clustering: by station
# ============================================================
ols_cluster_date = sm.OLS(y_train, X_train_sm).fit(
    cov_type="cluster",
    cov_kwds={"groups": date_groups}
)

# ============================================================
# --- TWO-WAY CLUSTERING (station + date) ---
# Statsmodels: pass Nx2 matrix
# ============================================================
groups_two_way = np.column_stack([station_groups, date_groups])

ols_cluster_tw = sm.OLS(y_train, X_train_sm).fit(
    cov_type="cluster",
    cov_kwds={"groups": groups_two_way}
)

# --- Print both for comparison ---
print("\n=== OLS with Quarter Clustering ===")
print(ols_cluster_date.summary())

print("\n=== OLS with Two-Way Clustering (Station + Date) ===")
print(ols_cluster_tw.summary())

Coefficients: [-1.77850166  0.90669861  0.12385409  1.57511016]
Intercept: 1.6127793112128612

=== OLS with Quarter Clustering ===
                            OLS Regression Results                            
Dep. Variable:           diff_in_temp   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.334
Method:                 Least Squares   F-statistic:                     159.5
Date:                Wed, 03 Dec 2025   Prob (F-statistic):           1.65e-11
Time:                        05:27:32   Log-Likelihood:                -735.55
No. Observations:                 433   AIC:                             1481.
Df Residuals:                     428   BIC:                             1501.
Df Model:                           4                                         
Covariance Type:              cluster                                         
                            coef    std err          z      P>|z|      [0.025  

/tmp/ipython-input-1798019389.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['NVDI_squared'] = X['mean_NDVI_transformed'] ** 2


#GMB for regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np

# --- Ensure data is sorted by date ---
# Adjust "DATE_X" if your date column has a different name
df_sorted = df_lr_cleaned_qr.sort_values("quarter_idx").copy()

# --- Define Features (X) and Target (y) ---
X = df_sorted[['mean_NDVI_transformed', 'HGHT', 'if_summer', 'NVDI_squared']]
y = df_sorted['diff_in_temp']

# --- TIME-BASED TRAIN–TEST SPLIT (BY DATE) ---
# Use unique dates so the split is on whole days, not random rows
unique_dates = df_sorted["quarter_idx"].sort_values().unique()
cutoff_idx = int(len(unique_dates) * 0.8)
cutoff_date = unique_dates[cutoff_idx]

train_mask = df_sorted["quarter_idx"] <= cutoff_date
test_mask  = df_sorted["quarter_idx"] > cutoff_date

X_train_gbm = X[train_mask]
X_test_gbm  = X[test_mask]
y_train_gbm = y[train_mask]
y_test_gbm  = y[test_mask]

print(f"Train period: {df_sorted.loc[train_mask, 'quarter_idx'].min()} → {df_sorted.loc[train_mask, 'quarter_idx'].max()}")
print(f"Test  period: {df_sorted.loc[test_mask, 'quarter_idx'].min()} → {df_sorted.loc[test_mask, 'quarter_idx'].max()}")

# --- SCALE FEATURES ---
scaler_gbm = StandardScaler()
X_train_scaled_gbm = scaler_gbm.fit_transform(X_train_gbm)
X_test_scaled_gbm = scaler_gbm.transform(X_test_gbm)

# --- GRADIENT BOOSTING REGRESSOR MODEL: the "SAFE" GBM ---
gbm_model = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=0.01,
    max_depth=2,          # a weak learner would suffice
    subsample=0.6,        # stochastic boosting
    min_samples_leaf=20,  # each terminal node has at least 20 training points
    random_state=42
)

# Train the model
gbm_model.fit(X_train_scaled_gbm, y_train_gbm)

# --- Make Predictions ---
y_pred_gbm = gbm_model.predict(X_test_scaled_gbm)

# --- Evaluate Model ---
r2 = r2_score(y_test_gbm, y_pred_gbm)
mse = mean_squared_error(y_test_gbm, y_pred_gbm)
rmse = np.sqrt(mse)

print(f"\nGBM Model Evaluation:")
print(f"R-squared: {r2:.4f}")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")

# Optional: Feature importances
feature_importances = pd.Series(gbm_model.feature_importances_, index=X.columns)
print("\nFeature Importances:")
formatted_importances = feature_importances.sort_values(ascending=False).apply(lambda x: '{:.4f}'.format(x))
print(formatted_importances)


Train period: 202001 → 202301
Test  period: 202302 → 202303

GBM Model Evaluation:
R-squared: 0.8978
Mean Squared Error: 0.1920
Root Mean Squared Error: 0.4382

Feature Importances:
NVDI_squared             0.8286
HGHT                     0.1595
mean_NDVI_transformed    0.0119
if_summer                0.0000
dtype: object


In [ ]:
y_pred_train = gbm_model.predict(X_train_scaled_gbm)
print("Train R²:", r2_score(y_train_gbm, y_pred_train))
print("Test  R²:", r2_score(y_test_gbm, y_pred_gbm))

Train R²: 0.905815196547818
Test  R²: 0.8977745578446117


#Interpretation


# **Interpretation: How NDVI Explains the Difference Between ERA5 Temperatures and Weather Station Data**

### **1. NDVI captures land surface characteristics that ERA5 cannot fully resolve**

ERA5 temperature is produced at coarse spatial resolution, while weather stations measure near-surface conditions directly affected by vegetation, soil moisture, shading, and surface roughness. NDVI represents vegetation density and greenness, which strongly influence localized heat exchange. Therefore, NDVI provides fine-scale information that helps explain the discrepancy between ERA5 and station observations.

---

### **2. Both linear and GBM models show that NDVI has a *nonlinear* effect**

In the linear model, **NDVI² is strongly significant**, while the linear NDVI term is negative. This pattern indicates an **inverted-U relationship**:

* At low NDVI, more vegetation increases the ERA5–station temperature difference.
* Beyond a threshold, higher NDVI reduces the temperature difference.

This implies that ERA5 systematically misrepresents temperature in areas with intermediate vegetation but aligns better in densely vegetated regions.

The GBM results reinforce this: **NDVI² contributes ~83% of the model’s predictive power**, dominating all other features. This confirms that the true relationship between vegetation and temperature bias is highly nonlinear.

---

### **3. NDVI explains microclimate-driven station–ERA5 discrepancies**

Vegetation modifies near-surface temperatures through evapotranspiration, shading, and changes in albedo. Stations located in vegetated environments experience localized cooling or warming that ERA5’s coarse grid cannot capture. As vegetation density increases, ERA5 errors change direction and magnitude, producing the observed curvature.

---

### **4. Model performance confirms NDVI’s explanatory value**

The time-based GBM achieves **R² ≈ 0.90**, showing NDVI (especially NDVI²) robustly explains a large share of ERA5–station temperature differences, even on future data.

---

### **Conclusion**

NDVI, particularly its nonlinear form, is a **key predictor** of temperature discrepancies between ERA5 and ground observations. It captures critical local vegetation effects missing from reanalysis data, enabling substantially more accurate bias correction.
